# Pre-Processing of POINT-seq data

## Loading Packages

In [1]:
import subprocess

## Download of POINT-seq example data

To demonstrate step-by-step the pre-processing of POINT-seq reads, a publicly available POINT-seq sample from [Sousa-Luís et al. Mol Cel. 2021](https://doi.org/10.1016/j.molcel.2021.02.034) will be used ([SRR12802839](https://www.ncbi.nlm.nih.gov/sra?term=SRX9271663)). This sample profiles a POINT-seq on untreated HeLa cells.

In [ ]:
sraString = "SRR12802839"
threads = 4
subprocess.run("fasterq-dump {} --threads {} --split-files".format(sraString, threads),
               shell=True, check=True)

2022-04-08T21:51:18 fasterq-dump.2.9.6 err: storage exhausted while writing file within file system module - system bad file descriptor error fd='19'
1900-01-00T00:00:00 fasterq-dump.2.9.6 err: storage exhausted while writing file within file system module - system bad file descriptor error fd='15'
2022-04-08T21:51:18 fasterq-dump.2.9.6 err: join_results_print().KFileWriteAll( at 2920284158 ) -> RC(rcFS,rcFile,rcWriting,rcStorage,rcExhausted)
2022-04-08T21:51:18 fasterq-dump.2.9.6 err: join_results_print().KFileWriteAll( at 2941255376 ) -> RC(rcFS,rcFile,rcWriting,rcStorage,rcExhausted)
2022-04-08T21:51:18 fasterq-dump.2.9.6 err: storage exhausted while writing file within file system module - system bad file descriptor error fd='12'
2022-04-08T21:51:18 fasterq-dump.2.9.6 err: storage exhausted while writing file within file system module - system bad file descriptor error fd='16'
2022-04-08T21:51:18 fasterq-dump.2.9.6 err: join_results_print().KFileWriteAll( at 2852126594 ) -> RC(rcFS

## Quality control - FastQC

## Trimming reads' adaptors

## Mapping the reads - STAR

### Create Index

### Mapping the reads